In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.latam.com/es_co/apps/personas/booking?fecha1_dia=31&fecha1_anomes=2020-12&auAvailability=1&ida_vuelta=ida&vuelos_origen=Bogot%C3%A1&from_city1=BOG&vuelos_destino=Ciudad%20de%20M%C3%A9xico&to_city1=MEX&flex=1&vuelos_fecha_salida_ddmmaaaa=11/09/2020&cabina=Y&nadults=2&nchildren=0&ninfants=0&cod_promo=&stopover_outbound_days=0&stopover_inbound_days=0&application=#/'

In [30]:
# To handle 403 error
#headers = {'User-Agent': 'Chrome/85.0.4183.83'}
#r = requests.get(url, headers=headers)

In [31]:
#r.status_code

In [32]:
#s = BeautifulSoup(r.text,'lxml')

In [6]:
# print(s.prettify())

## Importando selenium

# XPATH CheatSheet

http://labs.timtom.ch/library-webscraping/extras/xpath-cheatsheet.md.pdf

In [7]:
from selenium import webdriver

## 1. Instanciando el driver de Chrome

In [13]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(options=options)

## 2. Cargar la url en el navegador

In [14]:
driver.get(url)

## 3. Extraer la información de la página

In [16]:
vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
vuelos

[<selenium.webdriver.remote.webelement.WebElement (session="0226090b050980c2fe860ab1f60d3c25", element="b7d68702-93dc-45ea-a615-ab31a001855e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0226090b050980c2fe860ab1f60d3c25", element="3dcb7f44-27a4-4b56-852c-8f81c4ea4df0")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0226090b050980c2fe860ab1f60d3c25", element="bd878c95-acf5-4940-8744-263e0a60d2f6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0226090b050980c2fe860ab1f60d3c25", element="730cce0b-d5d6-465f-bb92-03ba6bd3d0b5")>]

In [18]:
vuelo = vuelos[0]
vuelo

<selenium.webdriver.remote.webelement.WebElement (session="0226090b050980c2fe860ab1f60d3c25", element="b7d68702-93dc-45ea-a615-ab31a001855e")>

### Hora de salida primer vuelo de la lista

In [20]:
vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')

'18:22'

### >>> Reto: 

Colocar la hora de llegada y la duración del vuelo

### Hora de llegada primer vuelo

In [21]:
vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')

'14:35'

### Duración del vuelo

In [40]:
vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

'PT21H13M'

In [41]:
boton_escalas = vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')
# Abrir el botón
boton_escalas.click()

In [43]:
segmentos = vuelo.find_elements_by_xpath('//div[@class="sc-hZSUBg gfeULV"]/div[@class="sc-cLQEGU hyoued"]')
segmentos

[<selenium.webdriver.remote.webelement.WebElement (session="0226090b050980c2fe860ab1f60d3c25", element="5f31e8e6-fc86-48f3-aaf6-b0597a1650ee")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0226090b050980c2fe860ab1f60d3c25", element="696990a5-9d3a-4978-8e33-c04007165748")>]

In [45]:
escalas = len(segmentos) - 1
escalas

1

In [46]:
segmento = segmentos[0]

In [50]:
# Aeropuerto
segmento.find_element_by_xpath('.//span[@class="sc-eTuwsz eumCTU"]/span[@class="sc-hXRMBi gVvErD"]').text

'El Dorado Intl.'

In [53]:
# Hora de salida del primer vuelo
segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/time').get_attribute('datetime')

'18:22'

---

### >>> Reto: 

1. Destino de cada escala
2. Hora de llegada de cada escala
3. Duración de cada vuelo
4. Duración de la escala
5. Número de vuelo
6. Modelo del avión

In [71]:
destinos = segmento.find_elements_by_xpath('//div[@class="sc-bwCtUz iybVbT"]/abbr')
print('>>> Destinos de cada escala:')
[destino.text for destino in destinos]

>>> Destinos de cada escala:


['BOG', 'LIM', 'LIM', 'MEX']

In [128]:
horas_llegada_por_escala = segmento.find_elements_by_xpath('//div[@class="sc-bwCtUz iybVbT"]/time')
print('>>> Hora llegada por cada escala:')
[hora.get_attribute('datetime') for hora in horas_llegada_por_escala][2]

>>> Hora llegada por cada escala:


'09:25'

In [142]:
duracion_por_vuelo = segmento.find_elements_by_xpath('//span[@class="sc-esjQYD dMquDU"]/time')
print('>>> Duracion por vuelo:')
duraciones = [duracion.get_attribute('datetime') for duracion in duracion_por_vuelo]
duraciones_vuelos = [duraciones[0] , duraciones[-1]]
duraciones_vuelos

>>> Duracion por vuelo:


['3:3', '6:10']

In [143]:
duracion_escalas = segmento.find_elements_by_xpath('//span[@class="sc-esjQYD dMquDU"]/time')
print('>>> Duracion por escala:')
[duracion.get_attribute('datetime') for duracion in duracion_escalas][1]

>>> Duracion por escala:


'12:0'

In [144]:
numeros_vuelos = segmento.find_elements_by_xpath('//div[@class="airline-flight-details"]/b')
print('>>> Número de vuelo:')
[numero_vuelo.text for numero_vuelo in numeros_vuelos]

>>> Número de vuelo:


['LA2447', 'LA2472']

In [147]:
modelo_aviones = segmento.find_elements_by_xpath('//div[@class="airline-flight-details"]/span[@class="sc-gzOgki uTyOl"]')
print('>>> Modelo Aviones:')
[modelo_avion.text for modelo_avion in modelo_aviones]

>>> Modelo Aviones:


['Airbus 320-200', 'Boeing 767-300']

### Cerrar el modal

In [150]:
driver.find_element_by_xpath('//div[@class="modal-content sc-iwsKbI eHVGAN"]//button[@class="close"]').click()

### Abrir precios

In [152]:
vuelo.click()

In [154]:
tarifas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class,"fare-")]')

---



### >>> Reto

1. Extraer el símbolo de la moneda
2. Extraer el precio de cada categoría

In [163]:
moneda = [
    tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
    for tarifa in tarifas]
moneda

['$', '$', '$']

In [162]:
precios = [tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text for tarifa in tarifas]
precios

['1.592.820', '1.903.940', '2.090.630']

## 4. Cerrar el navegador

In [165]:
#driver.close()